In [2]:
from oef.agents import Agent
import asyncio
from typing import List

from oef.agents import Agent
from oef.proxy import OEFNetworkProxy
from oef.query import Query

from oef.schema import DataModel, Description, AttributeSchema
from oef.query import Constraint, Eq

In [3]:
class GreetingsAgent(Agent):
    """A class that implements the greeting agent."""

    def on_message(self, msg_id: int, dialogue_id: int, origin: str, content: bytes):
        print("[{}]: Received message: msg_id={}, dialogue_id={}, origin={}, content={}"
              .format(self.public_key, msg_id, dialogue_id, origin, content))
        if content == b"hello":
            print("[{}]: Sending greetings message to {}".format(self.public_key, origin))
            self.send_message(1, dialogue_id, origin, b"greetings")
            self.stop()
        if content == b"greetings":
            self.stop()

    def on_search_result(self, search_id: int, agents: List[str]):
        if len(agents) > 0:
            print("[{}]: Agents found: {}".format(self.public_key, agents))
            for a in agents:
                self.send_message(0, 0, a, b"hello")
        else:
            print("[{}]: No agent found.".format(self.public_key))
            self.stop()


In [4]:
# create agents

client_proxy = OEFNetworkProxy("greetings_client", oef_addr="127.0.0.1", port=3333)
server_proxy = OEFNetworkProxy("greetings_server", oef_addr="127.0.0.1", port=3333)
client_agent = GreetingsAgent(client_proxy)
server_agent = GreetingsAgent(server_proxy)

In [5]:
# connect the agents to the OEF
await client_agent.async_connect()
await server_agent.async_connect()


True

In [6]:
# register the greetings service agent on the OEF
say_hello = AttributeSchema("say_hello", bool, True, "Set to 'True' if your service answers to 'hello' messages.")
greetings_model = DataModel("greetings", [say_hello], "Greetings service.")
greetings_description = Description({"say_hello": True}, greetings_model)
server_agent.register_service(0, greetings_description)

In [7]:
# the client executes the search for greetings services
# we are looking for services that answers to "hello" messages
query = Query([Constraint("say_hello", Eq(True))], greetings_model)

print("[{}]: Search for 'greetings' services. search_id={}".format(client_agent.public_key, 0))
client_agent.search_services(0, query)


[greetings_client]: Search for 'greetings' services. search_id=0


In [8]:
# run the agents
try:
    loop = asyncio.get_event_loop()
    await asyncio.gather(
        client_agent.async_run(),
        server_agent.async_run())
finally:
    client_agent.stop()
    server_agent.stop()

    await client_agent.async_disconnect()
    await server_agent.async_disconnect()


[greetings_client]: Agents found: ['greetings_server']
[greetings_server]: Received message: msg_id=0, dialogue_id=0, origin=greetings_client, content=b'hello'
[greetings_server]: Sending greetings message to greetings_client
[greetings_client]: Received message: msg_id=1, dialogue_id=0, origin=greetings_server, content=b'greetings'
